Development of the data pipeline for the raw measurements from Project Breathe

In [169]:
import pandas as pd
import src.data.helpers as dh
import src.data.breathe_data as breathe_data

In [170]:
path = (
    dh.get_path_to_main()
    + "DataFiles/BR/MeasurementData/Breathe_Spirometer_20231113.csv"
)
df = pd.read_csv(path)

In [171]:
print(df.columns)
print(df.shape)

Index(['UserId', 'EntityId', 'Timestamp', 'ClientTimestamp', 'IsDeleted',
       'CaptureType', 'CaptureDeviceMake', 'CaptureDeviceModel',
       'ThirdPartyId', 'PEF', 'PEFTime', 'FEV6', 'FEV1', 'FEV1PersonalBest',
       'FEV075', 'FEV1DivFEV6', 'FEV1DivFVC', 'FEF2575', 'FEV1Percent', 'EVol',
       'FVC', 'InvalidEntry', 'VolumeTimeCurve', 'FlowVolumeCurve',
       'IsCapturePrimary', 'CaptureSessionId'],
      dtype='object')
(104324, 26)


In [172]:
df.IsDeleted.unique()

array([False,  True])

In [173]:
tmp_size = df.shape[0]
df = df[~df.IsDeleted]
print(f"Removed {tmp_size - df.shape[0]} entries manually deleted on the app")

Removed 2565 entries manually deleted on the app


In [174]:
df.describe()

,EntityId,PEF,PEFTime,FEV6,FEV1,FEV1PersonalBest,FEV075,FEV1DivFEV6,FEV1DivFVC,FEF2575,FEV1Percent,EVol,FVC
count,1.017590e+05,87122.000000,4549.0,100680.000000,101759.000000,96319.000000,96319.000000,96319.000000,4163.000000,98277.000000,101176.000000,4549.0,7634.000000
mean,1.023662e+06,279.116710,0.0,3.029517,2.130394,3.262218,1.896338,0.681364,30.227024,1.631205,60.820797,0.0,1.116860
std,3.904265e+04,199.826929,0.0,1.068609,0.880368,0.979416,0.857896,0.176934,33.537712,1.169019,25.287741,0.0,1.790167
min,1.000001e+06,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
25%,1.002048e+06,147.000000,0.0,2.340000,1.560000,2.900000,1.420000,0.620000,0.000000,0.900000,45.000000,0.0,0.000000
50%,1.008223e+06,264.000000,0.0,2.770000,1.910000,3.500000,1.710000,0.690000,0.000000,1.190000,59.000000,0.0,0.000000
75%,1.029546e+06,435.000000,0.0,3.650000,2.620000,3.900000,2.350000,0.800000,61.700001,2.130000,77.000000,0.0,1.980000
max,1.336051e+06,966.000000,0.0,9.830000,6.030000,5.300000,5.780000,1.000000,100.000000,8.680000,700.000000,0.0,7.130000


In [175]:
def count_na(series):
    return series.isna().sum()


df.agg(count_na)

UserId                     0
EntityId                   0
Timestamp                  0
ClientTimestamp            0
IsDeleted                  0
CaptureType                0
CaptureDeviceMake       5946
CaptureDeviceModel      5977
ThirdPartyId          101151
PEF                    14637
PEFTime                97210
FEV6                    1079
FEV1                       0
FEV1PersonalBest        5440
FEV075                  5440
FEV1DivFEV6             5440
FEV1DivFVC             97596
FEF2575                 3482
FEV1Percent              583
EVol                   97210
FVC                    94125
InvalidEntry               0
VolumeTimeCurve        99802
FlowVolumeCurve        99801
IsCapturePrimary           0
CaptureSessionId           0
dtype: int64

In [176]:
df["Datetime recorded"] = pd.to_datetime(df.Timestamp, utc=False)

In [177]:
cols_to_keep = [
    "UserId",
    "Datetime recorded",
    "FEV1",
    "FEF2575",
    "PEF",
    "InvalidEntry",
]
df1 = df[cols_to_keep]

In [178]:
df1.head()

,UserId,Datetime recorded,FEV1,FEF2575,PEF,InvalidEntry
0,60d96f45-ec56-470b-89ae-0085c9f073a7,2022-08-04 17:15:31.378609200,2.73,2.53,326.0,False
1,60d96f45-ec56-470b-89ae-0085c9f073a7,2022-08-04 17:15:31.378609200,2.68,2.49,353.0,False
2,60d96f45-ec56-470b-89ae-0085c9f073a7,2022-08-04 17:15:31.378609200,2.70,2.34,351.0,False
3,60d96f45-ec56-470b-89ae-0085c9f073a7,2022-08-04 17:15:31.379609100,2.88,NaN,NaN,False
4,60d96f45-ec56-470b-89ae-0085c9f073a7,2022-08-02 15:25:26.475628300,2.57,2.52,333.0,False


In [179]:
def get_BR_ID_to_partition_key_mapping():
    df = pd.read_excel(
        dh.get_path_to_main() + "DataFiles/BR/PredModInputData.xlsx",
        sheet_name="brPatient",
        usecols="A, AD",
    )
    # Set ID as string
    df.ID = df.ID.astype(str)
    return df
df_key_map = get_BR_ID_to_partition_key_mapping()

In [180]:
# Is mapping complete?
keys_intersect = set(df1.UserId).intersection(set(df_key_map.PartitionKey))
print(f"{len(keys_intersect)} intersecting keys")
keys_missing_in_map = set(df1.UserId) - set(df_key_map.PartitionKey)
print(f"{len(keys_missing_in_map)} keys from physdata are missing in mapping table")
keys_missing_in_physdata = set(df_key_map.PartitionKey) - set(df1.UserId)
print(f"{len(keys_missing_in_physdata)} keys from mapping table are missing in phys data")

239 intersecting keys
215 keys from physdata are missing in mapping table
18 keys from mapping table are missing in phys data


In [181]:
df2 = df1.merge(df_key_map, left_on='UserId', right_on='PartitionKey', how='inner')
df2.ID.nunique()

240

In [182]:
# Why 1 off error??

In [183]:
df3 = df2.drop(columns=['UserId', 'PartitionKey'])

In [184]:
df3

,Datetime recorded,FEV1,FEF2575,PEF,InvalidEntry,ID
0,2022-08-04 17:15:02.545056800,2.44,1.18,0.0,False,183
1,2022-08-04 17:15:02.545056800,2.52,1.35,0.0,False,183
2,2022-08-04 17:15:02.546055300,2.52,1.35,0.0,False,183
3,2022-08-04 17:15:02.546055300,2.84,0.00,0.0,False,183
4,2022-08-04 17:15:02.546055300,2.82,1.53,0.0,False,183
...,...,...,...,...,...,...
76714,2022-08-10 05:30:16.027219400,3.41,3.04,483.0,True,273
76715,2022-08-10 05:30:16.027219400,3.27,2.97,492.0,False,273
76716,2022-08-10 05:31:57.706153700,3.29,3.15,471.0,True,273
76717,2022-08-10 05:32:50.942244600,3.31,3.09,472.0,False,273


In [185]:
# Understand invalid entry

df3.ID.value_counts()
df3["Date recorded"] = df3["Datetime recorded"].dt.date
df_for_ID = df3[df3.ID == '101'].reset_index()
df_for_ID.InvalidEntry.value_counts()

InvalidEntry
True     12275
False     3900
Name: count, dtype: int64

In [186]:
# Has invalid entry
df_for_ID[df_for_ID["Date recorded"] == df_for_ID["Date recorded"][0]]

,index,Datetime recorded,FEV1,FEF2575,PEF,InvalidEntry,ID,Date recorded
0,49269,2022-08-04 17:16:15.691332000,1.31,0.54,0.0,False,101,2022-08-04
1,49270,2022-08-04 17:16:15.692332100,1.31,0.57,0.0,False,101,2022-08-04
2,49271,2022-08-04 17:16:15.692332100,1.31,0.67,0.0,False,101,2022-08-04
3,49272,2022-08-04 17:16:15.692332100,1.30,0.69,0.0,False,101,2022-08-04
4,49273,2022-08-04 17:16:15.692332100,1.28,0.60,0.0,False,101,2022-08-04
...,...,...,...,...,...,...,...,...
5720,54989,2022-08-04 08:32:02.173501500,1.64,0.88,176.0,True,101,2022-08-04
5721,54990,2022-08-04 08:32:02.173501500,1.54,0.91,168.0,False,101,2022-08-04
5722,54991,2022-08-04 08:32:02.173501500,1.67,0.97,172.0,True,101,2022-08-04
5723,54992,2022-08-04 08:32:02.173501500,1.63,0.94,170.0,True,101,2022-08-04
